In [70]:
import requests
import pandas as pd

headers = {"User-agent": "email@email.com"}
ticker = 'pypl'

# SEC CIK end points return ticker

Return the CIK number for the relavent stock.

In [71]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC CIK list")


cik_matching_ticker(ticker)

'0001633917'

# Submissions Data

submission data end point: https://data.sec.gov/submissions/CIK##########.json



Return submission numbers of the SEC filings.

#### company_json catagories
- sic : The sector.
- sic description : The description of the sector.
- filings : The SEC filings the company submitted.
- fiscalYearEnd : THe end of the fiscal year for the company.

In [72]:
def get_submissions_for_ticker(tickers, headers=headers, only_filings_df=False):
    cik = cik_matching_ticker(tickers)
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    if only_filings_df:
        return pd.DataFrame(company_json["filings"]["recent"])
    return company_json

In [76]:
company_json = get_submissions_for_ticker(ticker,headers=headers, only_filings_df=False)

company_json['sic']
company_json['sicDescription']
company_json['fiscalYearEnd']

'1231'

#### Filtering the filings for 10-K and 10-Q

In [77]:
def get_filter_filing(ticker, ten_k=True, accession_number_only=False, headers=headers):
    company_filing_df = get_submissions_for_ticker(
        ticker, headers=headers, only_filings_df=True
    )
    if ten_k:
        df = company_filing_df[company_filing_df["form"] == "10-K"]
    else:
        df = company_filing_df[company_filing_df["form"] == "10-Q"]

    if accession_number_only:
        df = df.set_index("reportDate")
        return df["accessionNumber"]
    else:
        return df

In [171]:
filtered_filings= get_filter_filing(ticker, ten_k=False, accession_number_only=False, headers=headers)
filtered_filings_accession_only= get_filter_filing(ticker, ten_k=False, accession_number_only=True, headers=headers)
display(filtered_filings.head())
display(filtered_filings_accession_only)

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
17,0001633917-23-000156,2023-11-02,2023-09-30,2023-11-01T19:36:59.000Z,34,10-Q,001-36859,231369612,,19308724,1,1,pypl-20230930.htm,10-Q
35,0001633917-23-000117,2023-08-03,2023-06-30,2023-08-02T19:24:53.000Z,34,10-Q,001-36859,231137583,,16552932,1,1,pypl-20230630.htm,10-Q
66,0001633917-23-000072,2023-05-09,2023-03-31,2023-05-08T19:11:44.000Z,34,10-Q,001-36859,23899495,,15148780,1,1,pypl-20230331.htm,10-Q
104,0001633917-22-000167,2022-11-04,2022-09-30,2022-11-03T19:29:22.000Z,34,10-Q,001-36859,221359785,,17383114,1,1,pypl-20220930.htm,10-Q
120,0001633917-22-000137,2022-08-03,2022-06-30,2022-08-02T19:32:26.000Z,34,10-Q,001-36859,221130516,,17033494,1,1,pypl-20220630.htm,10-Q


reportDate
2023-09-30    0001633917-23-000156
2023-06-30    0001633917-23-000117
2023-03-31    0001633917-23-000072
2022-09-30    0001633917-22-000167
2022-06-30    0001633917-22-000137
2022-03-31    0001633917-22-000088
2021-09-30    0001633917-21-000169
2021-06-30    0001633917-21-000149
2021-03-31    0001633917-21-000100
2020-09-30    0001633917-20-000169
2020-06-30    0001633917-20-000134
2020-03-31    0001633917-20-000093
2019-09-30    0001633917-19-000210
2019-06-30    0001633917-19-000180
2019-03-31    0001633917-19-000118
2018-09-30    0001633917-18-000203
2018-06-30    0001633917-18-000171
2018-03-31    0001633917-18-000096
2017-09-30    0001633917-17-000171
2017-06-30    0001633917-17-000136
2017-03-31    0001633917-17-000075
2016-09-30    0001633917-16-000243
2016-06-30    0001633917-16-000203
2016-03-31    0001633917-16-000161
2015-09-30    0001633917-15-000052
2015-06-30    0001633917-15-000008
Name: accessionNumber, dtype: object

## Accessing Company Facts

Return `us_gaap_data` with company facts:
- `company_facts = get_facts_json(ticker, headers=headers)`
- `us_gaap_data = company_facts["facts"]["us-gaap"]`

`us_gaap_data [list]` : A list of dictionarys with:
- `key` = name of the financial marker
- `value` = `['label', 'description', 'units']`
    - `label [str]` = the name of the financial key
    - `description [str]` = the description of the financial key
    - `units [dict]` :
        - `key [str]` = the unit of the financial marker
        - `value [dict]`


In [79]:
def get_facts_json(ticker, headers=headers):
    cik = cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts



In [138]:
company_facts = get_facts_json(ticker, headers=headers)
us_gaap_data = company_facts["facts"]["us-gaap"]

fact = list(us_gaap_data.keys())
details = list(us_gaap_data.values())
i = 0

print(fact[i])
print('\n-----label-----')
print(details[i]['label'])
print(type(details[i]['label']))
print('-----label-----\n')

print('-----description-----')
print(details[i]['description'])
print(type(details[i]['description']))
print('-----description-----\n')

print('-----units-----')
print(details[i]['units'])
print(details[i]['units'].keys())
print(type(details[i]['units']))
print('-----units-----\n')

print('-----units - USD-----')
print(details[i]['units']['USD'])
print(type(details[i]['units']['USD']))
print('-----units - USD-----\n')

AccountsPayableAndOtherAccruedLiabilitiesCurrent

-----label-----
Accounts Payable and Other Accrued Liabilities, Current
<class 'str'>
-----label-----

-----description-----
Amount of liabilities incurred to vendors for goods and services received, and accrued liabilities classified as other, payable within one year or the normal operating cycle, if longer.
<class 'str'>
-----description-----

-----units-----
{'USD': [{'end': '2016-12-31', 'val': 192000000, 'accn': '0001633917-17-000171', 'fy': 2017, 'fp': 'Q3', 'form': '10-Q', 'filed': '2017-10-24', 'frame': 'CY2016Q4I'}, {'end': '2017-09-30', 'val': 974000000, 'accn': '0001633917-17-000171', 'fy': 2017, 'fp': 'Q3', 'form': '10-Q', 'filed': '2017-10-24', 'frame': 'CY2017Q3I'}]}
dict_keys(['USD'])
<class 'dict'>
-----units-----

-----units - USD-----
[{'end': '2016-12-31', 'val': 192000000, 'accn': '0001633917-17-000171', 'fy': 2017, 'fp': 'Q3', 'form': '10-Q', 'filed': '2017-10-24', 'frame': 'CY2016Q4I'}, {'end': '2017-09-30', 'val':

#### Return Company Fact to DF

In [162]:
def facts_to_df(ticker, headers=headers):
    company_facts = get_facts_json(ticker, headers=headers)
    us_gaap_data = company_facts["facts"]["us-gaap"]
    df_data = []

    for fact, details in us_gaap_data.items():
        for unit in details["units"].keys():
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}

    return df, labels_dict

In [169]:
facts, labels = facts_to_df(ticker, headers=headers)

display(facts.head())
print(labels)

,val,accn,fy,fp,form,filed,frame,fact,start
end,,,,,,,,,
2016-12-31,192000000.0,0001633917-17-000171,2017,Q3,10-Q,2017-10-24,CY2016Q4I,AccountsPayableAndOtherAccruedLiabilitiesCurrent,NaT
2017-09-30,974000000.0,0001633917-17-000171,2017,Q3,10-Q,2017-10-24,CY2017Q3I,AccountsPayableAndOtherAccruedLiabilitiesCurrent,NaT
2014-12-31,115000000.0,0001633917-15-000008,2015,Q2,10-Q,2015-07-29,NaN,AccountsPayableCurrent,NaT
2015-06-30,188000000.0,0001633917-15-000008,2015,Q2,10-Q,2015-07-29,CY2015Q2I,AccountsPayableCurrent,NaT
2015-09-30,114000000.0,0001633917-15-000052,2015,Q3,10-Q,2015-10-29,CY2015Q3I,AccountsPayableCurrent,NaT


{'AccountsPayableAndOtherAccruedLiabilitiesCurrent': 'Accounts Payable and Other Accrued Liabilities, Current', 'AccountsPayableCurrent': 'Accounts Payable, Current', 'AccountsPayableRelatedPartiesCurrentAndNoncurrent': 'Accounts Payable, Related Parties', 'AccountsReceivableNetCurrent': 'Accounts Receivable, after Allowance for Credit Loss, Current', 'AccountsReceivableRelatedParties': 'Accounts Receivable, Related Parties', 'AccruedIncomeTaxes': 'Accrued Income Taxes', 'AccruedLiabilitiesCurrent': 'Accrued Liabilities, Current', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment': 'Accumulated Depreciation, Depletion and Amortization, Property, Plant, and Equipment', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax': 'Accumulated Other Comprehensive Income (Loss), Net of Tax', 'AdditionalPaidInCapital': 'Additional Paid in Capital', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationAndExerciseOfStockOptions': 'APIC, Share-based Payment Arrangement, Rec

## Filter the company facts and labels

In [173]:
def annual_facts(ticker, headers=headers):
    accession_nums = get_filter_filing(
        ticker, ten_k=True, accession_number_only=True, headers=headers
    )
    df, labels = facts_to_df(ticker, headers=headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", index="end", columns="fact")
    pivot.rename(columns=labels, inplace=True)
    return pivot.T

pivot_annual = annual_facts(ticker, headers=headers)   

In [176]:
def quarterly_facts(ticker, headers=headers):
    accession_nums = get_filter_filing(
        ticker, ten_k=False, accession_number_only=True, headers=headers
    )
    df, labels = facts_to_df(ticker, headers=headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=['fact', 'end'], keep='last')
    pivot = ten_q.pivot_table(values="val", index="end", columns="fact")
    pivot.rename(columns=labels, inplace=True)
    return pivot.T

pivot_quarterly = quarterly_facts(ticker, headers=headers)


